In [1]:
%load_ext autotime

In [2]:
import boto3
import json
import time

from etltools import s3

time: 351 ms


In [3]:
aws_info = {
    'profile_name': 'sandbox',
}

uid = '1000101'

time: 1.26 ms


# Test ConfigInfo

In [11]:
from lambda_client.config_info import ConfigInfo

configs = ConfigInfo('lambda_client/lambda.cfg')

print configs.claims_bucket
print configs.claims_path
print
print configs.benefits_bucket
print configs.benefits_path
print
print configs.claims_table

picwell.sandbox.analytics
junghoon/lambda_calculator

picwell.sandbox.analytics
junghoon/lambda_calculator_benefits

ma_claims
time: 3.33 ms


In [12]:
all_states = configs.all_states

print '{} states'.format(len(all_states))
print all_states

51 states
['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.48 ms


# Test ClaimsClient

In [ ]:
from lambda_client import ClaimsClient

In [ ]:
# Test S3:
client = ClaimsClient(aws_info, 
                      s3_bucket=configs.claims_bucket,
                      s3_path=configs.claims_path)

client.get(uid)

In [ ]:
# Test DynamoDB:
client = ClaimsClient(aws_info,
                      table_name=configs.claims_table)

client.get(uid)

In [ ]:
# Test configuration file:
client = ClaimsClient(aws_info)

client.get(uid)

# Test BenefitsClient

In [13]:
from lambda_client import BenefitsClient

time: 897 µs


In [14]:
client = BenefitsClient(aws_info)

print client.all_states

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 78.2 ms


In [5]:
plans = client._get_one_state('01')
print '{} plans read for state 01'.format(len(plans))

plans = client._get_one_state('04')
print '{} plans read for state 04'.format(len(plans))

47 plans read for state 01
75 plans read for state 04
time: 577 ms


In [6]:
plans = client.get_by_state(['01', '04'])
print '{} plans read'.format(len(plans))

122 plans read
time: 306 ms


In [7]:
plans = client.get_all()
print '{} plans read'.format(len(plans))

3558 plans read
time: 2.88 s


In [8]:
# Compare the timing against reading the entire file:
from lambda_client.storage_utils import _read_json

session = boto3.Session(**aws_info)
resource = session.resource('s3')

time: 57.5 ms


In [9]:
all_plans = _read_json('picwell.sandbox.medicare', 'ma_benefits/cms_2018_pbps_20171005.json', resource)

print '{} plans read'.format(len(plans))

3558 plans read
time: 4.37 s


In [10]:
# Ensure that the same plans are read:
sort_key = lambda plan: plan['picwell_id']
assert sorted(all_plans, key=sort_key) == sorted(plans, key=sort_key)

time: 131 ms


# Test Cost Breakdown

In [ ]:
from test_client import LambdaCalculatorTestClient

client = LambdaCalculatorTestClient(aws_info)

In [ ]:
pids = ['2820028008119', '2820088001036']

client.calculate_breakdown(uid, pids)

# Test Batch Calculation

In [ ]:
uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

print '{} uids read'.format(len(uids))

In [ ]:
uids[:10]

In [ ]:
requests_per_second = 100

for uid in uids:
#     client.calculate_async(uid, months=['01'])
    client.calculate_async(uid)
    time.sleep(1.0/requests_per_second)  